In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optimizers
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

np.random.seed(123)
torch.manual_seed(123)

In [2]:
# データの準備
path = "D:Statistics/data/deep_leraning/"
root = os.path.join(path, "torch", "mnist")
transform = transforms.Compose([transforms.ToTensor(), lambda x: x.view(-1)])
mnist_train = datasets.MNIST(root=root, download=True, train=True, transform=transform)
mnist_test = datasets.MNIST(root=root, download=True, train=False, transform=transform)

train_dataloader = DataLoader(mnist_train, batch_size=100, shuffle=True)
test_dataloader = DataLoader(mnist_test, batch_size=100, shuffle=False)

In [12]:
# DNNを定義
class DNN1(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.l1 = nn.Linear(input_dim, hidden_dim)
        self.a1 = nn.ReLU()
        self.l2 = nn.Linear(hidden_dim, hidden_dim)
        self.a2 = nn.ReLU()
        self.l3 = nn.Linear(hidden_dim, hidden_dim)
        self.a3 = nn.ReLU()
        self.l4 = nn.Linear(hidden_dim, output_dim)
        self.layers = [self.l1, self.a1, self.l2, self.a2, self.l3, self.a3, self.l4]

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
class DNN2(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.l1 = nn.Linear(input_dim, hidden_dim)
        self.a1 = nn.LeakyReLU(0.1)
        self.l2 = nn.Linear(hidden_dim, hidden_dim)
        self.a2 = nn.LeakyReLU(0.1)
        self.l3 = nn.Linear(hidden_dim, hidden_dim)
        self.a3 = nn.LeakyReLU(0.1)
        self.l4 = nn.Linear(hidden_dim, output_dim)
        self.layers = [self.l1, self.a1, self.l2, self.a2, self.l3, self.a3, self.l4]

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [13]:
# モデルの学習
# アルゴリズムの定義
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classes = np.unique(train_dataloader.dataset.classes).shape[0]
m = np.prod(train_dataloader.dataset.data.shape[1:])
model = DNN2(m, 200, classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optimizers.SGD(model.parameters(), lr=0.01)

def compute_loss(t, y):
    return criterion(y, t)

def train_step(x, t):
    model.train()
    preds = model(x)
    loss = compute_loss(t, preds)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss, preds

# アルゴリズムの設定
epochs = 30

In [14]:
# 確率的勾配法でモデルを学習
for epoch in range(epochs):
    train_loss = 0.0
    train_acc = 0.0

    for (x, t) in train_dataloader:
        x, t = x.to(device), t.to(device)
        loss, preds = train_step(x, t)
        train_loss += loss.item()
        train_acc += accuracy_score(t.tolist(), preds.argmax(dim=-1).tolist())

    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataloader)
    print("epoch: {}, loss: {:.3}, acc: {:,.3f}".format(epoch+1, train_loss, train_acc))

epoch: 1, loss: 2.28, acc: 0.211
epoch: 2, loss: 1.99, acc: 0.553
epoch: 3, loss: 0.824, acc: 0.776
epoch: 4, loss: 0.487, acc: 0.860
epoch: 5, loss: 0.4, acc: 0.884
epoch: 6, loss: 0.361, acc: 0.897
epoch: 7, loss: 0.335, acc: 0.904
epoch: 8, loss: 0.314, acc: 0.910
epoch: 9, loss: 0.296, acc: 0.915
epoch: 10, loss: 0.28, acc: 0.920
epoch: 11, loss: 0.264, acc: 0.924
epoch: 12, loss: 0.251, acc: 0.928
epoch: 13, loss: 0.238, acc: 0.932
epoch: 14, loss: 0.227, acc: 0.934
epoch: 15, loss: 0.215, acc: 0.938
epoch: 16, loss: 0.205, acc: 0.941
epoch: 17, loss: 0.196, acc: 0.944
epoch: 18, loss: 0.187, acc: 0.946
epoch: 19, loss: 0.179, acc: 0.949
epoch: 20, loss: 0.172, acc: 0.951
epoch: 21, loss: 0.165, acc: 0.953
epoch: 22, loss: 0.158, acc: 0.954
epoch: 23, loss: 0.152, acc: 0.956
epoch: 24, loss: 0.145, acc: 0.958
epoch: 25, loss: 0.14, acc: 0.960
epoch: 26, loss: 0.134, acc: 0.961
epoch: 27, loss: 0.129, acc: 0.962
epoch: 28, loss: 0.124, acc: 0.964
epoch: 29, loss: 0.12, acc: 0.966
e

In [15]:
# モデルの評価
def test_step(x, t):
    model.eval()
    preds = model(x)
    loss = criterion(preds, t)
    return loss, preds

test_loss = 0.0
test_acc = 0.0

for (x, t) in test_dataloader:
    x, t = x.to(device), t.to(device)
    loss, preds = test_step(x, t)
    test_loss += loss.item()
    test_acc += accuracy_score(t.tolist(), preds.argmax(dim=-1).tolist())

test_loss /= len(test_dataloader)
test_acc /= len(test_dataloader)
print('test_loss: {:.3f}, test_acc: {:.3f}'.format(test_loss, test_acc))

test_loss: 0.122, test_acc: 0.964
